In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.optimizers import *
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

from nltk.tokenize import sent_tokenize
from konlpy.corpus import kolaw
from konlpy.tag import Okt
import matplotlib.pyplot as plt

from keras import models,layers,optimizers,metrics,preprocessing

import os

import re

path='C:/Users/student/Downloads/데이터들/'

Using TensorFlow backend.


In [2]:
chatbotData=pd.read_csv(path+'ChatData.csv')
question, answer = list(chatbotData['Q']), list(chatbotData['A'])

In [3]:
len(answer)

11823

In [4]:
question=question[:100]
answer=answer[:100]

In [5]:
for i in range(10):
    print("질문:"+question[i])
    print('답변:'+answer[i])
    print()

질문:12시 땡!
답변:하루가 또 가네요.

질문:1지망 학교 떨어졌어
답변:위로해 드립니다.

질문:3박4일 놀러가고 싶다
답변:여행은 언제나 좋죠.

질문:3박4일 정도 놀러가고 싶다
답변:여행은 언제나 좋죠.

질문:PPL 심하네
답변:눈살이 찌푸려지죠.

질문:SD카드 망가졌어
답변:다시 새로 사는 게 마음 편해요.

질문:SD카드 안돼
답변:다시 새로 사는 게 마음 편해요.

질문:SNS 맞팔 왜 안하지ㅠㅠ
답변:잘 모르고 있을 수도 있어요.

질문:SNS 시간낭비인 거 아는데 매일 하는 중
답변:시간을 정하고 해보세요.

질문:SNS 시간낭비인데 자꾸 보게됨
답변:시간을 정하고 해보세요.



In [6]:
PAD = "<PADDING>" # 패딩
STA = "<START>" # 시작
END = "<END>" # 끝
OOV = "<OOV>" # out of vocabulary

PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

ENCODER_INPUT = 0
DECODER_INPUT = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 갯수
maxSequences = 30

# 임베딩 벡터 차원
embeddingDim = 100

# LSTM 출력 차원
lstmHiddenDim = 128

# 정규표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

In [7]:
def posTag(sentences):
    
    tagger = Okt()
    
    sentencePos=[]
    for sentence in sentences:
        # 특수문자 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        sentence = " ".join(tagger.morphs(sentence))
        sentencePos.append(sentence)
    
    return sentencePos

In [8]:
question = posTag(question)
answer = posTag(answer)

In [9]:
# 질문 + 대답 문장을 하나로 합치기
sentences=[]
sentences.extend(question)
sentences.extend(answer)
len(sentences)

200

In [10]:
# 단어 배열 생성
words=[]
for sentence in sentences:
    for word in sentence.split():
        words.append(word)
        
len(words)

966

In [11]:
len(words)

966

In [12]:
# words에서 길이가 0인 단어를 삭제
words = [word for word in words if len(word)>0]

# 중복단어 삭제
words = list(set(words))
len(words)

450

In [13]:
len(words)

450

In [14]:
words[:0] = [PAD, STA, END, OOV]
words[:20]

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '관계',
 '반',
 '갑작스러웠나',
 '한테',
 '싫어',
 '살찐',
 '나',
 '됐으면',
 '모르고',
 '다음',
 '나온거',
 '절약',
 '기관',
 '여행',
 '행복',
 '남겨야']

In [15]:
# 단어에 대한 인덱스를 부여 -> 딕셔너리
wordToIndex = {word:index for index, word in enumerate(words)}
indexToWord = {index:word for index, word in enumerate(words)}
indexToWord

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '관계',
 5: '반',
 6: '갑작스러웠나',
 7: '한테',
 8: '싫어',
 9: '살찐',
 10: '나',
 11: '됐으면',
 12: '모르고',
 13: '다음',
 14: '나온거',
 15: '절약',
 16: '기관',
 17: '여행',
 18: '행복',
 19: '남겨야',
 20: '콕',
 21: '그럴',
 22: '개인',
 23: '집',
 24: '기름',
 25: '곳',
 26: '편해요',
 27: '먹어야지',
 28: '개념',
 29: '시켜야지',
 30: '진창',
 31: '업무',
 32: '갈거야',
 33: '야',
 34: '끄고',
 35: '매일',
 36: '서로',
 37: '하자고',
 38: '가야',
 39: '가끔',
 40: '드립니다',
 41: '버렸어',
 42: '괜찮아요',
 43: '첫인상',
 44: '시켜',
 45: '땡',
 46: '가',
 47: '온',
 48: '되',
 49: '이야기',
 50: '아는데',
 51: '이랑',
 52: '불',
 53: '했길',
 54: '비',
 55: '예요',
 56: '같',
 57: '애가',
 58: '마세요',
 59: '까지',
 60: '어서',
 61: '방학',
 62: '싫다',
 63: '같아',
 64: '들어올',
 65: '또',
 66: '친구',
 67: '되겠네요',
 68: '걔',
 69: '됨',
 70: '없어',
 71: '돌아가서',
 72: '가만',
 73: '켜고',
 74: '줘',
 75: 'PPL',
 76: '감',
 77: '된',
 78: '1',
 79: '궁금해',
 80: '가서',
 81: '바라요',
 82: '하세요',
 83: '키워',
 84: '잠깐',
 85: '두',
 86: '스트레스',
 87: '옆',
 88: '갑자기',

In [16]:
# 전처리
# 문장 -> 인덱스로 변환

In [17]:
# 문장 -> 인덱스로 변환
def convertTextToIndex(sentences, voc, mytype):
    
    sentencesIndex=[]
    for sentence in sentences:
        
        sentenceIndex=[]
        if mytype == DECODER_INPUT:
            
            sentenceIndex.extend([voc[STA]])
            
        for word in sentence.split():
            
            if voc.get(word) is not None: # 단어에 해당하는 인덱스가 있는 경우
                sentenceIndex.extend([voc[word]]) # 단어에 해당되는 인덱스가 추가
                
            else: # 사전에 없는 단어의 경우 OOV추가
                sentenceIndex.extend([voc[OOV]])
                
        if mytype == DECODER_TARGET:
            
            # 디코더 출력은 맨 마지막에 end 추가
            if maxSequences <= len(sentenceIndex):
                sentenceIndex = sentenceIndex[:maxSequences-1] + [voc[END]]
                
            else:
                sentenceIndex += [voc[END]]
                
        else:
            if len(sentenceIndex) > maxSequences:
                sentenceIndex = sentenceIndex[:maxSequences]
                
        # 0으로 채움(pad_sequence)
        sentenceIndex += [wordToIndex[PAD]] * (maxSequences-len(sentenceIndex))
        
        sentencesIndex.append(sentenceIndex)
            
    return np.asarray(sentencesIndex)

In [18]:
# 인코더 입력, 디코더 입력, 디코더 출력 -> 인덱스 변환
xEncoder = convertTextToIndex(question, wordToIndex, ENCODER_INPUT)
print(xEncoder[0])

xDecoder = convertTextToIndex(answer, wordToIndex, DECODER_INPUT)
print(xDecoder[0])

yDecoder = convertTextToIndex(answer, wordToIndex, DECODER_TARGET)
print(yDecoder[0])

[384  45   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[  1 268  46  65 242   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[268  46  65 242   2   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]


In [19]:
# 인코더 입력 : 12시 떙
# 디코더 입력 : START 하루 가   또   가네요
# 디코더 출력 : 하루   가  또 가네요  END

In [27]:
len(yDecoder),maxSequences, len(words)
# 100(답변 개수), 30(최대 단어 개수), 454(전체 단어 집합 개수)
oneHotData=np.zeros((len(yDecoder),maxSequences,len(words)))
oneHotData.shape

(100, 30, 454)

In [28]:
for i, seq in enumerate(yDecoder): # 100, 30, 454
    for j, index in enumerate(seq):
        oneHotData[i,j,index]=1
yDecoder=oneHotData

In [30]:
yDecoder[0].shape

(30, 454)

In [50]:
#훈련 모델 생성
#인코더 정의

#입력 문장의 인덱스 sequence를 입력
encoderInputs=layers.Input(shape=(None,))
#임베딩 계층
encoderOutputs=layers.Embedding(len(words),embeddingDim)(encoderInputs)

encoderOutputs,stateH, stateC=layers.LSTM(lstmHiddenDim,return_state=True, 
            dropout=0.2, recurrent_dropout=0.5)(encoderOutputs)
#return_state=True => 상태값 리턴
#LSTM은 2개 상태 존재(셀, 히든 스테이트)

encoderStates=[stateH, stateC]


#디코더 정의
#출력 문장의 인덱스 sequence를 입력
decoderInputs=layers.Input(shape=(None,))
#임베딩 계층
decoderEmbedding=layers.Embedding(len(words),
                                embeddingDim)
decoderOutputs=decoderEmbedding(decoderInputs)



decoderLSTM=layers.LSTM(lstmHiddenDim,
                        return_state=True, 
            return_sequences=True, 
                        dropout=0.2, 
                        recurrent_dropout=0.5)
decoderOutputs, _, _=decoderLSTM(decoderOutputs,initial_state=encoderStates)
decoderDense=layers.Dense(len(words), 
                          activation="softmax")
decoderOutputs=decoderDense(decoderOutputs)


In [51]:
model=models.Model([encoderInputs, decoderInputs],decoderOutputs)

In [52]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [53]:
#예측 모델 인코더 정의
encoderModel=models.Model(encoderInputs, 
                          encoderStates)

#예측 모델 디코더 정의
#바로 앞에 있는 디코더의 출력(상태)을 입력 받아서
#예측을 해야 함.
decoderStateInputH=layers.Input(shape=(lstmHiddenDim,))
decoderStateInputC=layers.Input(shape=(lstmHiddenDim,))
decoderStatesInputs=[decoderStateInputH,decoderStateInputC]

#임베딩 계층
decoderOutputs=decoderEmbedding(decoderInputs)
#LSTM 계층
decoderOutputs, stateH, stateC=decoderLSTM(decoderOutputs,
           initial_state=decoderStatesInputs)
decoderStates=[stateH, stateC]

#Dense계층을 통해 원핫 형식으로 예측 단어 인덱스를 추출
decoderOutputs=decoderDense(decoderOutputs)

#예측 모델 디코더 설정
decoderModel=models.Model([decoderInputs]+decoderStatesInputs,
            [decoderOutputs]+decoderStates)

In [70]:
#인덱스를 문장으로 변환
def convertIndexToText(indexs, voc):
    #구현
    sentence=""
    for i in indexs:
        if i==END_INDEX: #종료 인덱스
            break;
        if voc.get(i) is not None:
            sentence+=voc[i]
        else:
            sentence.extend([voc[OOV_INDEX]])
        sentence +=" "    
    return sentence    

In [71]:
#에폭
for epoch in range(10):
    print("total epoch:", epoch+1)
    history=model.fit([xEncoder,xDecoder], yDecoder,
             epochs=100,
             batch_size=64,
                     verbose=0)
    print("accuracy :", history.history['accuracy'])
    print("loss :", history.history['loss'])
    #문장 예측
    #3박 4일 놀러 가고 싶다 -> 여행 은 언제나 좋죠
    
    inputEncoder=xEncoder[2].reshape(1,xEncoder[2].shape[0]) #(30,) ->(1,30)
    inputDecoder=xDecoder[2].reshape(1,xDecoder[2].shape[0]) #(30,) ->(1,30)
    
    results=model.predict([inputEncoder,inputDecoder])
    
    #결과값에 대해서 가장 큰 값의 위치를 구함
    index=np.argmax(results[0], 1)
    #인덱스 -> 문장으로 변환
    sentence=convertIndexToText(index, indexToWord)
    print(sentence)
    print()

total epoch: 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
accuracy : [0.27733332, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7866667, 0.787, 0.7873333, 0.7863333, 0.7876667, 0.78833336, 0.79033333, 0.7873333, 0.79033333, 0.7923333, 0.79066664, 0.79033333, 0.78966665, 0.791, 0.7926667, 0.78933334, 0.795, 0.795, 0.795, 0.79766667, 0.796, 0.8016667, 0.7926667, 0.797, 0.7966667, 0.79866666, 0.8053333, 0.80466664, 0.8043333, 0.8016667, 0.8013333, 0.80733335, 0.805, 0.806, 0.80366665, 0.8066667, 0.811, 0.8106667, 0.81133336, 0.8146667, 0.8183333, 0.81133336, 0.814, 0.814, 0.817, 0.8183333, 0.819, 0.818, 0.81666666, 0.823, 0.82033336, 0.819, 0.8183333, 0.818, 0.8233333, 0.82166666, 0.82133335, 0.8243333, 0.82266665, 0.8236667, 0.824, 0.8233333, 0.8246667, 0.824, 0.826, 0.825, 0.82766664, 0.82566667, 0.82533336, 0.82533336, 0.82666665, 0.82566667, 0.824, 0.828, 0.8

accuracy : [0.9573333, 0.9586667, 0.95633334, 0.958, 0.959, 0.9623333, 0.96033335, 0.95933336, 0.96133333, 0.96033335, 0.962, 0.96033335, 0.961, 0.9626667, 0.95966667, 0.96166664, 0.963, 0.9623333, 0.96133333, 0.963, 0.9626667, 0.9623333, 0.963, 0.96433336, 0.96466666, 0.965, 0.9623333, 0.9636667, 0.965, 0.96466666, 0.9633333, 0.9623333, 0.96533334, 0.963, 0.964, 0.965, 0.96466666, 0.966, 0.964, 0.96466666, 0.966, 0.966, 0.96666664, 0.96566665, 0.966, 0.966, 0.96533334, 0.96666664, 0.966, 0.9673333, 0.9673333, 0.96533334, 0.967, 0.96666664, 0.966, 0.968, 0.9676667, 0.967, 0.9683333, 0.96666664, 0.967, 0.968, 0.9683333, 0.9676667, 0.968, 0.9673333, 0.968, 0.96933335, 0.9683333, 0.969, 0.96666664, 0.9676667, 0.9683333, 0.9683333, 0.969, 0.96933335, 0.96966666, 0.96933335, 0.9683333, 0.9673333, 0.96966666, 0.97, 0.9683333, 0.969, 0.9683333, 0.96966666, 0.969, 0.968, 0.96933335, 0.96966666, 0.9676667, 0.96933335, 0.97033334, 0.97033334, 0.96933335, 0.9686667, 0.97033334, 0.96966666, 0.9686

accuracy : [0.9716667, 0.9713333, 0.9716667, 0.972, 0.9716667, 0.9713333, 0.9723333, 0.9716667, 0.9713333, 0.9723333, 0.9716667, 0.9716667, 0.9723333, 0.97066665, 0.9713333, 0.9713333, 0.9716667, 0.972, 0.9713333, 0.971, 0.97066665, 0.9723333, 0.97033334, 0.971, 0.971, 0.9716667, 0.972, 0.972, 0.9716667, 0.9716667, 0.97066665, 0.9713333, 0.9716667, 0.9723333, 0.9716667, 0.9713333, 0.97066665, 0.9723333, 0.97066665, 0.9716667, 0.972, 0.971, 0.9713333, 0.972, 0.9723333, 0.971, 0.9723333, 0.9713333, 0.9723333, 0.972, 0.972, 0.9716667, 0.9716667, 0.9716667, 0.9726667, 0.9716667, 0.9726667, 0.9716667, 0.9716667, 0.9713333, 0.9716667, 0.971, 0.971, 0.97, 0.97, 0.971, 0.97033334, 0.97, 0.9716667, 0.9716667, 0.9726667, 0.9713333, 0.97066665, 0.971, 0.9723333, 0.971, 0.9716667, 0.9723333, 0.97333336, 0.97066665, 0.973, 0.974, 0.972, 0.973, 0.9726667, 0.9723333, 0.971, 0.9726667, 0.9713333, 0.97333336, 0.97033334, 0.9723333, 0.9723333, 0.97, 0.9716667, 0.9726667, 0.972, 0.972, 0.97366667, 0.9723

accuracy : [0.97366667, 0.975, 0.97433335, 0.97466666, 0.97466666, 0.97466666, 0.975, 0.976, 0.97466666, 0.9763333, 0.976, 0.97533333, 0.97433335, 0.97533333, 0.9766667, 0.97333336, 0.977, 0.976, 0.9766667, 0.9763333, 0.973, 0.97533333, 0.97466666, 0.975, 0.97433335, 0.97433335, 0.975, 0.975, 0.97366667, 0.9763333, 0.97533333, 0.975, 0.97566664, 0.9766667, 0.975, 0.976, 0.9766667, 0.9763333, 0.9773333, 0.97566664, 0.9763333, 0.97333336, 0.97566664, 0.97566664, 0.9763333, 0.97466666, 0.9766667, 0.975, 0.97566664, 0.977, 0.97566664, 0.9766667, 0.9766667, 0.97566664, 0.9766667, 0.9763333, 0.97566664, 0.9773333, 0.9773333, 0.975, 0.97433335, 0.97433335, 0.9776667, 0.976, 0.9766667, 0.976, 0.978, 0.97433335, 0.975, 0.976, 0.97566664, 0.975, 0.976, 0.97566664, 0.97533333, 0.97566664, 0.97466666, 0.9763333, 0.9766667, 0.975, 0.977, 0.977, 0.977, 0.9763333, 0.976, 0.9776667, 0.9766667, 0.975, 0.9766667, 0.97866666, 0.97533333, 0.9763333, 0.97466666, 0.97566664, 0.9773333, 0.97533333, 0.9766667

In [72]:
# 문장 입력
def makePredictInput(sentence):
    sentences=[]
    sentences.append(sentence)
    sentences=posTag(sentences)
    inputSeq=convertTextToIndex(sentences,wordToIndex, ENCODER_INPUT)
    return inputSeq

In [73]:
#예측 답변 생성
def generateText(inputSeq):
        #입력을 인코더에 넣고, 마지막 상태 구함
        states=encoderModel.predict(inputSeq)
        
        #목표 시퀀스 초기화
        targetSeq=np.zeros((1,1))
        #<START> 시그널을 추가
        targetSeq[0,0]=STA_INDEX
        #인덱스 초기화
        indexs=[]
        
        #디코더 반복
        while 1:
            decoderOuputs, stateH, stateC=decoderModel.predict([targetSeq]+states)
            #결과를 원핫인코딩 형식으로 변환
            index=np.argmax(decoderOuputs[0,0,:])
            indexs.append(index)
            #종료 체크
            if index==END_INDEX or len(indexs)>=maxSequences:
                break
                
                #targetSeq를 이전 출력으로 설정
            targetSeq=np.zeros((1,1))
            targetSeq[0,0]=index    #STA_INDEX 
            
            #디코더의 이전 상태를 다음 디코더 예측에 사용
            states=[stateH, stateC]
            
        #인덱스를 문장으로 변환
        sentence=convertIndexToText(indexs, indexToWord)
        
        return sentence                         
        
    
    

In [74]:
inputSeq=makePredictInput("3박4일 같이 놀러가고 싶다")
#[[320, 157, ...... 19,0,0,0,0...0]]

In [75]:
sentence=generateText(inputSeq)
sentence

'여행 은 언제나 좋죠 '